In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
path = "/home/araichev/transit_flows/maps/data/hamilton_gtfs_20171113.zip"
#print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


In [ ]:
dates = feed.get_week(51)
feed.compute_route_stats(dates)

In [ ]:
freq = "6H"
sts = feed.compute_stop_time_series(dates, freq=freq, split_directions=True)
rts = feed.compute_route_time_series(trip_stats, dates, freq=freq)
fts = feed.compute_feed_time_series(trip_stats, dates, freq=freq, split_route_types=True)

In [ ]:
fts

In [ ]:
(
    rts
    .pipe(gt.unstack_time_series)
    .merge(feed.routes.filter(["route_id", "route_type"]), how="left")
    .groupby(["datetime", "indicator", "route_type"])
    .agg({"value": lambda x: x.sum(skipna=True, min_count=1)})  # Preserve NaNs
    .reset_index("datetime")
#     .fillna(-1)
#     .pivot_table(
#         index=["datetime"], columns=["indicator", "route_type"]
#     )
#     .value
#     .replace({-1: np.nan})
)

In [ ]:
# Slicing 
display(fts.loc[:, ("num_trips", slice(None))])
display(fts.xs("num_trips", axis="columns"))

In [ ]:
f = fts = feed.compute_feed_time_series(trip_stats, dates[:1], freq="12H", split_route_types=False)
display(f)
g = gt.unstack_time_series(f)

# columns = [c for c in g.columns if c not in ["datetime", "value"]]
# h = g.pivot_table(index="datetime", columns=columns).value.sort_index(
#     axis="columns"
# )

# hours = (h.index[1] - h.index[0]).components.hours
# if hours != 0:
#     freqy = f"{hours}H"
# else:
#     freqy = "D"

# print(freqy)

h = gt.restack_time_series(g)
h

In [ ]:
f.index

In [ ]:
delta.components.hours

In [ ]:
feed1 = feed.copy()
cal = feed1.calendar.copy()
cal["monday"] = 0
feed1.calendar = cal
feed1.compute_feed_time_series(trip_stats, dates, freq="12H")

In [ ]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()